In [1]:
pwd()

"/Users/nmayhall/code/FermiCG-data/cr-qubit"

In [2]:
using Pkg; Pkg.activate("../../FermiCG/")
using FermiCG, NPZ
readdir()

  Activating project at `~/code/FermiCG`


13-element Vector{String}:
 ".ipynb_checkpoints"
 "C_act_loc_sorted_3.molden"
 "cr_qubit_cmf_5clusters.jld2"
 "cr_qubit_cmf_bst1.ipynb"
 "cr_qubit_cmf_bst2.ipynb"
 "cr_qubit_cmf_tpsci2.ipynb"
 "cr_qubit_scf.ipynb"
 "cr_qubit_scf_integrals_h0.npz.npy"
 "cr_qubit_scf_integrals_h1.npz.npy"
 "cr_qubit_scf_integrals_h2.npz.npy"
 "cr_qubit_scf_mo_coeffs.npz.npy"
 "excitations1.xlsx"
 "~\$excitations1.xlsx"

In [4]:
using JLD2

@load "cr_qubit_cmf_5clusters_bases_M20.jld2"

6-element Vector{Symbol}:
 :ints
 :clusters
 :init_fspace
 :da1
 :db1
 :cluster_bases

In [6]:
clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, da1, db1);

In [20]:
using LinearAlgebra

nroots = 3
ci_vector = FermiCG.TPSCIstate(clusters, FermiCG.FockConfig(init_fspace), R=nroots);
# Add the lowest energy single exciton to basis
ci_vector[FermiCG.FockConfig(init_fspace)][FermiCG.ClusterConfig([2,1,1,1,1])] = zeros(Float64,nroots)
ci_vector[FermiCG.FockConfig(init_fspace)][FermiCG.ClusterConfig([3,1,1,1,1])] = zeros(Float64,nroots)
# FermiCG.eye!(ci_vector)
# FermiCG.set_vectors!(ci_vector, Matrix{Float64}(I,size(ci_vector)))
FermiCG.randomize!(ci_vector)
FermiCG.orthonormalize!(ci_vector)
display(ci_vector, root=1)

 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =     3  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 1.000               3                   ( 5,5 )( 3,3 )( 3,3 )( 3,3 )( 3,3 )
 --------------------------------------------------


In [22]:
e0, v0 = FermiCG.tpsci_ci(ci_vector, cluster_ops, clustered_ham,
                            thresh_cipsi=1e-3, # Threshold for adding to P-space
                            thresh_foi=1e-5,    # Threshold for keeping terms when defining FOIS
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            matvec=3);

 ci_vector     : (3, 3)
 thresh_cipsi  : 0.001
 thresh_foi    : 1.0e-5
 thresh_asci   : -1
 thresh_var    : -1.0
 max_iter      : 10
 conv_thresh   : 0.0001
 nbody         : 4
 incremental   : true
 ci_conv       : 1.0e-5
 ci_max_iter   : 50
 ci_max_ss_vecs: 12
 davidson      : false
 max_mem_ci    : 20.0
 matvec        : 3


     Selected CI Iteration:    1 epsilon:   0.00100000
 Memory needed to hold full CI matrix:   0.00000007 (Gb)

 |== Tensor Product State CI =======================================
 Hamiltonian matrix dimension =     3: 
 Build full Hamiltonian matrix with dimension:       0.000458 seconds (2.54 k allocations: 78.203 KiB)
 Diagonalize: 
                                      0.000041 seconds (11 allocations: 1.844 KiB)
 Compute S2 expectation values:                      0.001065 seconds (1.62 k allocations: 63.656 KiB)
  Root       Energy           S2
     1 -105.91574232   2.00000000
     2 -105.83948254   0.00000000
     3 -105.83791692   0.00000000
 ==========

In [25]:
@time e2 = FermiCG.compute_pt2_energy(v0, cluster_ops, clustered_ham, thresh_foi=1e-8)



 |........................do batched PT2............................
 thresh_foi    :1.0e-8
 prescreen     :true
 H0            :Hcmf
 nbody         :4
 Norms of input states:
   1.00000000
   1.00000000
   1.00000000
 Compute <0|H0|0>:                                   0.753712 seconds (10.85 M allocations: 330.793 MiB)
 Compute <0|H|0>:                                    5.711675 seconds (330.63 M allocations: 9.442 GiB, 31.76% gc time)
 Number of jobs:    16781
 Number of threads: 24
   |----------------------------------------------------------------------------------------------------|
   |0%                                                                                              100%|
   |---------------------------------------------------------------------------------------------------|
 Time spent computing E2        266.6 (s)
  Root         E(0)         E(2)
     1 -105.92861190 -105.92958306
     2 -105.85377820 -105.85484094
     3 -105.85335774 -105.85434669
 .........

3-element Vector{Float64}:
 -0.0009711608796984458
 -0.0010627395651977953
 -0.0009889554511517938